In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns

# Load and Explore the Data

In [ ]:
df = pd.read_csv("/kaggle/input/deep-learning-az-ann/Churn_Modelling.csv")
df

# Explored the dataset using

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe().T

# Checked for missing values using

In [ ]:
df.isnull().any()

# Checked duplicates using

In [ ]:
df.duplicated()

In [ ]:
df.dtypes

In [ ]:
df.columns

# Convert Categorical Columns to Numerical Columns

In [ ]:
df.drop(['RowNumber', 'CustomerId','Surname'], axis=1, inplace=True, errors='ignore')

In [ ]:
categorical_cols =  ['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited']
for col in categorical_cols:
    if col in df.columns:
        df[col] = pd.Categorical(df[col]).codes

In [ ]:
df.dtypes

# Delete unimportant columns

# Plot correlation matrix

In [ ]:

corr_matrix = df.corr()
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True, 
            linewidths=0.5, linecolor='white', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix')
plt.show()

# Split data into training and testing sets

In [ ]:

X = df.drop('Exited', axis=1)  # features
y = df['Exited']  # target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)


# Define the ANN model architecture

In [ ]:

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



# Compile the model

In [ ]:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=7, validation_data=(X_test, y_test))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.3f}')

# Plot training and validation loss,Plot training and validation accuracy

In [ ]:

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Show plot
plt.show()